In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error, r2_score

In [5]:
# Load the dataset
# file_path = "C:/Users/tarek/Downloads/m_and_a/data_eur/07_Gross_income_1.xlsx"
file_path = "C:/Users/tarek/Downloads/project_ma/merged_training_data_side_by_side 2.xlsx"
xls = pd.ExcelFile(file_path)
df = pd.read_excel(xls, sheet_name='Sheet1')

In [11]:
# Columns to ignore
ignore_columns = [
    'USABLE_FOR', 'INDUSTRY_CODE', 'SECTOR_CODE', 'DEAL_SIZE', 'OMG_RATIO', 'SECTION',
    'USABLE_FOR.1', 'INDUSTRY_CODE.1', 'SECTOR_CODE.1', 'DEAL_SIZE.1', 'OMG_RATIO.1', 'SECTION.1'
]

# Select features (X) from SELL SIDE
X_columns = [col for col in df.columns[:len(df.columns)//2] if col not in ignore_columns]
X = df[X_columns]

# Select target (y) from BUY SIDE
y_columns = [col for col in df.columns[len(df.columns)//2:] if col not in ignore_columns]
y = df[y_columns]

# Extract only numerical columns from 2019 to 2023 for both X and y
X_numerical = X[['SELL_DEC 2019', 'SELL_DEC 2020', 'SELL_DEC 2021', 'SELL_DEC 2022']].astype(float)
y_numerical = y[['BUY_DEC 2019']].astype(float)

In [7]:
print(X_columns)

['N', 'SELL_ANNOUNCE_DATE', 'SELL_TARGET', 'SELL_NAME', 'SELL_CURRENCY', 'SELL_INDUSTRY_CODE', 'SELL_SECTOR_CODE', 'SELL_OMG_RATIO', 'SELL_DEC 2019', 'SELL_DEC 2020', 'SELL_DEC 2021', 'SELL_DEC 2022']


In [131]:
X_numerical.shape, y_numerical.shape

((3686, 4), (3686, 1))

In [12]:
# Combine both DataFrames side by side
combined_df = pd.concat([X_numerical, y_numerical], axis=1)
combined_df=combined_df.dropna()
combined_df.corr()

,SELL_DEC 2019,SELL_DEC 2020,SELL_DEC 2021,SELL_DEC 2022,BUY_DEC 2019
SELL_DEC 2019,1.000000,0.988706,0.982541,0.985979,0.181218
SELL_DEC 2020,0.988706,1.000000,0.993523,0.993209,0.178317
SELL_DEC 2021,0.982541,0.993523,1.000000,0.997490,0.184773
SELL_DEC 2022,0.985979,0.993209,0.997490,1.000000,0.188265
BUY_DEC 2019,0.181218,0.178317,0.184773,0.188265,1.000000


In [133]:
# Define X (features) and y (target)
X = combined_df[['DEC 2019', 'DEC 2020', 'DEC 2021', 'DEC 2022']]
y = combined_df[['DEC 2023']]

KeyError: "None of [Index(['DEC 2023'], dtype='object')] are in the [columns]"

In [ ]:
X_numerical=X_numerical[:2905]
print(X_numerical.shape)
y_numerical = y_numerical[:2905]
print(y_numerical.shape)

        DEC 2019    DEC 2020    DEC 2021    DEC 2022    DEC 2023
0      45.197100   41.663350   15.948880   36.992340   19.631040
1     134.976425  158.306234  376.345258  543.896738  615.795786
2      -0.007100    0.349315    0.444288    0.559482    0.337463
3       0.699360    0.177100    0.089817    0.479000    0.372052
4       8.324802  -42.341068  -10.301725  -23.925621  -63.988526
...          ...         ...         ...         ...         ...
2900    2.599000    3.243000    2.987000    2.866000    3.014000
2901   15.754780   13.623800   12.414900   13.923000    2.113020
2902    0.202000    0.782000    1.095000    0.382000    0.024000
2903    2.888000    3.170000    3.602000    3.037000    2.581000
2904   -0.293200   -0.307990    0.081312    0.100352    0.271794

[2905 rows x 5 columns]
      DEC 2023.1
0        478.418
1       8735.530
2         -6.177
3      10481.000
4        284.000
...          ...
3122      67.893
3123     138.838
3124      -6.177
3125     975.000
3126    

In [113]:
# Split data into training and test sets (80% training, 20% validation)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [114]:
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(2948, 4) (738, 4) (2948, 1) (738, 1)


In [86]:
import pandas as pd
import numpy as np
from tsfresh import extract_features
from tsfresh.feature_extraction import ComprehensiveFCParameters, MinimalFCParameters
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.model_selection import RandomizedSearchCV
import scipy.stats as st

### 1. Feature Extraction Using tsfresh ###

# Convert X_train to long format for tsfresh
X_train_long = X_train.copy()
X_train_long["id"] = np.arange(len(X_train))  # Assign a unique ID to each row
X_train_long = X_train_long.melt(id_vars=["id"], var_name="time", value_name="value")

# Extract features
extracted_features = extract_features(X_train_long, column_id="id", column_sort="time",
                                      default_fc_parameters=MinimalFCParameters())

# # Remove NaN features
# extracted_features = extracted_features.dropna(axis=1)
print(extracted_features)

Feature Extraction: 100%|██████████| 20/20 [00:08<00:00,  2.31it/s]

      value__sum_values  value__median  value__mean  value__length  \
0              1.207000       0.308000     0.301750            4.0   
1             50.137490      12.737520    12.534372            4.0   
2             42.520810      11.110975    10.630203            4.0   
3             21.849710       5.595925     5.462427            4.0   
4             -0.751565      -0.016043    -0.187891            4.0   
...                 ...            ...          ...            ...   
2717         234.578170      53.346135    58.644543            4.0   
2718           7.457000       2.401000     1.864250            4.0   
2719          63.799471      16.140933    15.949868            4.0   
2720          -7.633950       0.007475    -1.908487            4.0   
2721           6.006120       1.296135     1.501530            4.0   

      value__standard_deviation  value__variance  value__root_mean_square  \
0                      0.019992         0.000400                 0.302412   
1    

In [87]:
print(extracted_features.isna().sum())

value__sum_values            0
value__median                0
value__mean                  0
value__length                0
value__standard_deviation    0
value__variance              0
value__root_mean_square      0
value__maximum               0
value__absolute_maximum      0
value__minimum               0
dtype: int64


In [90]:
# Ensure extracted_features has the same index as X_train
extracted_features.index = X_train.index  # Align index

In [ ]:
# Merge extracted features with original dataset
X_train_enhanced = pd.concat([X_train, extracted_features], axis=1)
print(X_train_enhanced)

       DEC 2019   DEC 2020   DEC 2021    DEC 2022  value__sum_values  \
3677   0.319000   0.271000   0.297000    0.320000           1.207000   
1421  11.988300   9.949250  13.486740   14.713200          50.137490   
2566   6.865460  10.076750  12.145200   13.433400          42.520810   
1341   5.160870   4.441300   6.030980    6.216560          21.849710   
1169  -1.511000   0.569145   0.791520   -0.601230          -0.751565   
...         ...        ...        ...         ...                ...   
1158  21.511300  31.214550  75.477720  106.374600         234.578170   
1196   2.925000   2.980000   1.877000   -0.325000           7.457000   
1374  15.990041  14.404842  17.112762   16.291826          63.799471   
897   -7.682220   0.014950   0.000000    0.033320          -7.633950   
3430   1.027950   1.086300   1.505970    2.385900           6.006120   

      value__median  value__mean  value__length  value__standard_deviation  \
3677       0.308000     0.301750            4.0          

In [92]:
X_train_enhanced.isna().sum()

DEC 2019                     0
DEC 2020                     0
DEC 2021                     0
DEC 2022                     0
value__sum_values            0
value__median                0
value__mean                  0
value__length                0
value__standard_deviation    0
value__variance              0
value__root_mean_square      0
value__maximum               0
value__absolute_maximum      0
value__minimum               0
dtype: int64

In [ ]:
# Repeat for X_test
X_test_long = X_test.copy()
X_test_long["id"] = np.arange(len(X_test))
X_test_long = X_test_long.melt(id_vars=["id"], var_name="time", value_name="value")

extracted_features_test = extract_features(X_test_long, column_id="id", column_sort="time",
                                           default_fc_parameters=MinimalFCParameters())

# # Drop NaN columns and match train columns
# extracted_features_test = extracted_features_test.dropna(axis=1)
# Reindex test features to match train features (fill missing ones with 0)
extracted_features_test = extracted_features_test.reindex(columns=X_train_enhanced.columns[1:], fill_value=0)


Feature Extraction: 100%|██████████| 20/20 [00:04<00:00,  4.35it/s]


In [94]:
print(extracted_features_test)

     DEC 2020  DEC 2021  DEC 2022  value__sum_values  value__median  \
0           0         0         0            5.77085       1.399175   
1           0         0         0            4.41397       1.044635   
2           0         0         0         6288.31000    1464.030000   
3           0         0         0           73.83039      19.643405   
4           0         0         0          130.46339      32.172275   
..        ...       ...       ...                ...            ...   
676         0         0         0          322.34943      80.410430   
677         0         0         0          109.14543      26.835690   
678         0         0         0           31.20291       7.997035   
679         0         0         0            4.79447       1.159435   
680         0         0         0          685.06859     171.480970   

     value__mean  value__length  value__standard_deviation  value__variance  \
0       1.442712            4.0                   0.328138     1.076

In [95]:
extracted_features_test.index = X_test.index  # Align index

In [ ]:
X_test_enhanced = pd.concat([X_test, extracted_features_test], axis=1)
print(X_test_enhanced)

        DEC 2019   DEC 2020    DEC 2021    DEC 2022  DEC 2020  DEC 2021  \
304      1.14187    1.10500     1.65648     1.86750         0         0   
1591     0.72090    0.87040     1.21887     1.60380         0         0   
2245  1068.00000 -468.35000  1860.06000  3828.60000         0         0   
451     21.59784   19.33425    12.94574    19.95256         0         0   
500     29.36644   29.49670    34.84785    36.75240         0         0   
...          ...        ...         ...         ...       ...       ...   
2623    66.46787   77.29390    83.52696    95.06070         0         0   
2528    26.60655   24.48255    27.06483    30.99150         0         0   
3536     5.87044    7.21055     8.78352     9.33840         0         0   
2410     0.66750    0.87380     1.44507     1.80810         0         0   
2222   173.48681  180.62415   169.47513   161.48250         0         0   

      DEC 2022  value__sum_values  value__median  value__mean  value__length  \
304          0     

In [ ]:
# Remove duplicate columns by keeping only the first occurrence
X_train_enhanced = X_train_enhanced.loc[:, ~X_train_enhanced.columns.duplicated()]
X_test_enhanced = X_test_enhanced.loc[:, ~X_test_enhanced.columns.duplicated()]

# Ensure test features match train features (fix column mismatch)
X_test_enhanced = X_test_enhanced.reindex(columns=X_train_enhanced.columns, fill_value=0)

# Now apply StandardScaler
scaler_X = StandardScaler()
scaler_y = StandardScaler()

X_train_scaled = scaler_X.fit_transform(X_train_enhanced)
print(X_train_scaled.shape)
X_test_scaled = scaler_X.transform(X_test_enhanced)
print(X_test_scaled.shape)

y_train_scaled = scaler_y.fit_transform(y_train)
print(y_train_scaled.shape)
y_test_scaled = scaler_y.transform(y_test)
print(y_test_scaled.shape)

(2722, 14)
(681, 14)
(2722, 1)
(681, 1)


In [ ]:
### 3. Hyperparameter Tuning Using RandomizedSearchCV ###
param_dist = {
    "n_neighbors": [1, 2, 3, 4, 5],  # Randomly choose from 3 to 20 neighbors
    "weights": ["uniform", "distance"],
    "metric": ["euclidean", "manhattan", "minkowski"],
    "p": [1, 2]
}

# Initialize KNN
knn = KNeighborsRegressor()

# Perform Randomized Search with 5-fold CV
random_search = RandomizedSearchCV(knn, param_distributions=param_dist, n_iter=200, 
                                   cv=10, scoring="r2", verbose=1, 
                                   n_jobs=-1, random_state=42)

# Fit the model
random_search.fit(X_train_scaled, y_train_scaled)

# Best hyperparameters
print("Best Parameters:", random_search.best_params_)

# Get the best model
best_knn = random_search.best_estimator_

### 4. Predict and Evaluate the Best KNN Model ###
# Predict on the test set
y_pred_scaled = best_knn.predict(X_test_scaled)
y_pred = scaler_y.inverse_transform(y_pred_scaled)
y_test_original = scaler_y.inverse_transform(y_test_scaled)

# Evaluate Model Performance
mae = mean_absolute_error(y_test_original, y_pred)
r2 = r2_score(y_test_original, y_pred)

print(f"Best KNN MAE: {mae}")
print(f"Best KNN R² Score: {r2}")

# ### 5. Show Actual vs Predicted Values ###
# results = pd.DataFrame({
#     "Actual DEC 2019": y_test_original[:, 0],
#     "Predicted DEC 2019": y_pred[:, 0],
#     "Actual DEC 2020": y_test_original[:, 1],
#     "Predicted DEC 2020": y_pred[:, 1],
#     "Actual DEC 2021": y_test_original[:, 2],
#     "Predicted DEC 2021": y_pred[:, 2],
#     "Actual DEC 2022": y_test_original[:, 3],
#     "Predicted DEC 2022": y_pred[:, 3],
#     "Actual DEC 2023": y_test_original[:, 4],
#     "Predicted DEC 2023": y_pred[:, 4],
# })
# print(results)

c:\Users\tarek\Downloads\CodeChallenge-ML-Engineer-main\CodeChallenge-ML-Engineer-main\myenv\lib\site-packages\sklearn\model_selection\_search.py:305: UserWarning: The total space of parameters 60 is smaller than n_iter=200. Running 60 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Fitting 10 folds for each of 60 candidates, totalling 600 fits
Best Parameters: {'weights': 'distance', 'p': 1, 'n_neighbors': 4, 'metric': 'euclidean'}
Best KNN MAE: 53.90655446630941
Best KNN R² Score: 0.6766353250247927


In [84]:
# Ensure y_test_original and y_pred are 2D arrays
if y_test_original.ndim == 1:
    y_test_original = y_test_original.reshape(-1, 1)

if y_pred.ndim == 1:
    y_pred = y_pred.reshape(-1, 1)

# Print shapes for debugging
print("y_test_original shape:", y_test_original.shape)
print("y_pred shape:", y_pred.shape)

# Now construct the DataFrame
results = pd.DataFrame({
    "Actual": y_test_original[:, 0],
    "Predicted": y_pred[:, 0],
})
print(results)

y_test_original shape: (681, 1)
y_pred shape: (681, 1)
        Actual    Predicted
0       49.931      67.0204
1       38.912      59.2492
2     5200.000   14965.1976
3    22399.170   37797.8130
4      224.553    7034.7510
..         ...          ...
676   2346.330  254341.2378
677     30.560     978.4008
678     33.169      58.1430
679     67.914     183.0050
680    115.807     495.3848

[681 rows x 2 columns]


In [115]:
# Normalize the data
scaler_X = StandardScaler()
scaler_y = StandardScaler()

X_train_scaled = scaler_X.fit_transform(X_train)
X_test_scaled = scaler_X.transform(X_test)

y_train_scaled = scaler_y.fit_transform(y_train)
y_test_scaled = scaler_y.transform(y_test)

In [29]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_absolute_error, r2_score
import numpy as np
import pandas as pd

# Define KNN Regressor
knn_regressor = KNeighborsRegressor(n_neighbors=5, weights="distance", metric="euclidean")

In [35]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.neighbors import KNeighborsRegressor
import scipy.stats as st

# Define hyperparameter distributions
param_dist = {
    "n_neighbors": st.randint(1, 20),  # Randomly choose from 3 to 20 neighbors
    "weights": ["uniform", "distance"],
    "metric": ["euclidean", "manhattan", "minkowski"],
    "p": [1, 2]
}

# Initialize KNN Regressor
knn = KNeighborsRegressor()

# Perform Randomized Search with 5-fold Cross-Validation
random_search = RandomizedSearchCV(knn, param_distributions=param_dist, n_iter=20, 
                                   cv=5, scoring="neg_mean_absolute_error", verbose=1, n_jobs=-1, random_state=42)

# Fit the model
random_search.fit(X_train_scaled, y_train_scaled)

# Best hyperparameters
print("Best Parameters:", random_search.best_params_)

# Get best model
best_knn = random_search.best_estimator_

# Predict on the test set
y_pred_scaled = best_knn.predict(X_test_scaled)
y_pred = scaler_y.inverse_transform(y_pred_scaled)

# Evaluate the model
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Best KNN MAE: {mae}")
print(f"Best KNN R² Score: {r2}")

Fitting 5 folds for each of 20 candidates, totalling 100 fits
Best Parameters: {'metric': 'manhattan', 'n_neighbors': 18, 'p': 2, 'weights': 'distance'}
Best KNN MAE: 36273.94785827254
Best KNN R² Score: -1.184354421856766


In [36]:
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsRegressor

# Define hyperparameter grid
param_grid = {
    "n_neighbors": [3, 5, 7, 10, 15],
    "weights": ["uniform", "distance"],
    "metric": ["euclidean", "manhattan", "minkowski"],
    "p": [1, 2]  # Only used when metric is 'minkowski'
}

# Initialize KNN Regressor
knn = KNeighborsRegressor()

# Perform Grid Search with 5-fold Cross-Validation
grid_search = GridSearchCV(knn, param_grid, cv=5, scoring="neg_mean_absolute_error", verbose=1, n_jobs=-1)

# Fit the model
grid_search.fit(X_train_scaled, y_train_scaled)

# Best hyperparameters
print("Best Parameters:", grid_search.best_params_)

# Get best model
best_knn = grid_search.best_estimator_

# Predict on the test set
y_pred_scaled = best_knn.predict(X_test_scaled)
y_pred = scaler_y.inverse_transform(y_pred_scaled)

# Evaluate the model
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Best KNN MAE: {mae}")
print(f"Best KNN R² Score: {r2}")

Fitting 5 folds for each of 60 candidates, totalling 300 fits
Best Parameters: {'metric': 'euclidean', 'n_neighbors': 15, 'p': 1, 'weights': 'distance'}
Best KNN MAE: 38242.055144575585
Best KNN R² Score: -1.4173407798189028


In [30]:
# Train the model
knn_regressor.fit(X_train_scaled, y_train_scaled)

KNeighborsRegressor(metric='euclidean', weights='distance')

In [31]:
# Predict on the test set
y_pred_scaled = knn_regressor.predict(X_test_scaled)

In [32]:
# Inverse transform the predictions to the original scale
y_pred = scaler_y.inverse_transform(y_pred_scaled)
y_test_original = scaler_y.inverse_transform(y_test_scaled)

In [33]:
# Evaluate model performance
mae = mean_absolute_error(y_test_original, y_pred)
r2 = r2_score(y_test_original, y_pred)

print(f"Mean Absolute Error (MAE): {mae}")
print(f"R² Score: {r2}")

# Create a DataFrame to compare actual vs predicted values
results = pd.DataFrame({
    "Actual DEC 2019": y_test_original[:, 0],
    "Predicted DEC 2019": y_pred[:, 0],
    "Actual DEC 2020": y_test_original[:, 1],
    "Predicted DEC 2020": y_pred[:, 1],
    "Actual DEC 2021": y_test_original[:, 2],
    "Predicted DEC 2021": y_pred[:, 2],
    "Actual DEC 2022": y_test_original[:, 3],
    "Predicted DEC 2022": y_pred[:, 3],
    "Actual DEC 2023": y_test_original[:, 4],
    "Predicted DEC 2023": y_pred[:, 4],
})

print(results)

Mean Absolute Error (MAE): 40265.50418517577
R² Score: -3.9335915560509753
    Actual DEC 2019  Predicted DEC 2019  Actual DEC 2020  Predicted DEC 2020  \
0        236817.569           10.928122        76023.111            7.745547   
1             3.850        89202.354560            5.691        91726.045916   
2           447.894         3928.556774          447.453         9491.846384   
3          3191.427         1626.122194         2139.522         3074.440110   
4           703.000        23027.833982          702.000        18363.224422   
..              ...                 ...              ...                 ...   
66          125.840           16.487058          170.490           10.898847   
67          674.244         5166.003341          631.327        10682.075356   
68         1745.472        43363.559504         2059.747        46918.398931   
69           55.466         3226.449273           65.945         3707.000622   
70         5580.286        20490.643814      

In [ ]:
# Train the XGBoost Regressor model
xgb_model = XGBRegressor(n_estimators=600, learning_rate=0.0001, max_depth=10, random_state=42, )
xgb_model.fit(X_train_scaled, y_train_scaled)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.0001, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=10, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=600, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=42, ...)

In [27]:
# Predict on the test set
y_pred_scaled = xgb_model.predict(X_test_scaled)

In [28]:
# Inverse transform the predictions
y_pred = scaler_y.inverse_transform(y_pred_scaled.reshape(-1, 5))

# Evaluate model performance
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# Display results
print("Mean Absolute Error (MAE):", mae)
print("R² Score:", r2)

# Show actual vs. predicted values
results = pd.DataFrame({
    "Actual DEC 2019": y_test.values[:, 0],
    "Predicted DEC 2019": y_pred[:, 0],
    "Actual DEC 2020": y_test.values[:, 1],
    "Predicted DEC 2020": y_pred[:, 1],
    "Actual DEC 2021": y_test.values[:, 2],
    "Predicted DEC 2021": y_pred[:, 2],
    "Actual DEC 2022": y_test.values[:, 3],
    "Predicted DEC 2022": y_pred[:, 3],
    "Actual DEC 2023": y_test.values[:, 4],
    "Predicted DEC 2023": y_pred[:, 4],
})
print(results)

Mean Absolute Error (MAE): 46481.08160484156
R² Score: -1.1027124576897223
    Actual DEC 2019  Predicted DEC 2019  Actual DEC 2020  Predicted DEC 2020  \
0        236817.569        42322.742188        76023.111        28882.871094   
1             3.850        44495.617188            5.691        29838.621094   
2           447.894        40734.351562          447.453        39455.878906   
3          3191.427        42322.742188         2139.522        28882.871094   
4           703.000       208164.656250          702.000       109461.625000   
..              ...                 ...              ...                 ...   
66          125.840        42322.742188          170.490        28882.871094   
67          674.244        28289.187500          631.327        30354.292969   
68         1745.472        26314.472656         2059.747        22491.601562   
69           55.466        42322.742188           65.945        28882.871094   
70         5580.286        44495.617188      

In [26]:
import optuna
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split

# Define an objective function for Optuna
def objective(trial):
    # Suggest values for hyperparameters
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 600),
        'learning_rate': trial.suggest_float('learning_rate', 0.0001, 0.1),
        'max_depth': trial.suggest_int('max_depth', 5, 24),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 12),
        'subsample': trial.suggest_float('subsample', 0.7, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.7, 1.0),
        'gamma': trial.suggest_float('gamma', 0, 5),
        'lambda': trial.suggest_float('lambda', 1, 10),
        'alpha': trial.suggest_float('alpha', 0, 10),
    }

    # Train the model
    model = XGBRegressor(**params, random_state=42)
    model.fit(X_train_scaled, y_train_scaled)

    # Predict and calculate MAE
    y_pred_scaled = model.predict(X_test_scaled)
    y_pred = scaler_y.inverse_transform(y_pred_scaled.reshape(-1, 5))
    
    return mean_absolute_error(y_test, y_pred)

# Run Optuna to find the best hyperparameters
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=40)

# Print the best parameters
best_params = study.best_params
print("Best Hyperparameters:", best_params)

# Train final model with the best parameters
xgb_model = XGBRegressor(**best_params, random_state=42)
xgb_model.fit(X_train_scaled, y_train_scaled)

# Predict on test set
y_pred_scaled = xgb_model.predict(X_test_scaled)
y_pred = scaler_y.inverse_transform(y_pred_scaled.reshape(-1, 5))

# Evaluate final model
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Final MAE:", mae)
print("Final R² Score:", r2)

[I 2025-01-30 17:27:00,916] A new study created in memory with name: no-name-34bca15e-18ab-4d44-a501-3ecff2fcfd75
[I 2025-01-30 17:27:01,524] Trial 0 finished with value: 49880.70287303724 and parameters: {'n_estimators': 406, 'learning_rate': 0.032227657827331516, 'max_depth': 20, 'min_child_weight': 2, 'subsample': 0.9169107333515552, 'colsample_bytree': 0.7220966376272845, 'gamma': 0.3034024126060919, 'lambda': 8.124641798806213, 'alpha': 4.715928866248303}. Best is trial 0 with value: 49880.70287303724.
[I 2025-01-30 17:27:02,145] Trial 1 finished with value: 49482.04489032241 and parameters: {'n_estimators': 478, 'learning_rate': 0.0690604392027491, 'max_depth': 14, 'min_child_weight': 12, 'subsample': 0.7738349900768076, 'colsample_bytree': 0.8481107798942644, 'gamma': 1.7443841882049176, 'lambda': 9.656479016198944, 'alpha': 8.037781619290708}. Best is trial 1 with value: 49482.04489032241.
[I 2025-01-30 17:27:03,108] Trial 2 finished with value: 50464.491965316905 and parameter

Best Hyperparameters: {'n_estimators': 571, 'learning_rate': 0.08916677726566508, 'max_depth': 19, 'min_child_weight': 5, 'subsample': 0.858721031496248, 'colsample_bytree': 0.850079314644279, 'gamma': 4.940554853719812, 'lambda': 7.661064349998632, 'alpha': 8.98994500225716}
Final MAE: 46481.08160484156
Final R² Score: -1.1027124576897223


In [137]:
import pandas as pd

def add_prefix_to_indicators(file_path, output_path):
    # Load the CSV file
    df = pd.read_excel(file_path)
    
    # Rename section columns for clarity
    df.rename(columns={'SECTION': 'SELL_SIDE_SECTION', 'SECTION.1': 'BUY_SIDE_SECTION'}, inplace=True)
    
    # Identify SELL-SIDE and BUY-SIDE rows
    sell_side_mask = df["SELL_SIDE_SECTION"].notna()
    buy_side_mask = df["BUY_SIDE_SECTION"].notna()
    
    # Apply prefixes to financial indicators based on the section type
    df.loc[sell_side_mask, "SELL_SIDE_SECTION"] = "sell_" + df.loc[sell_side_mask, "SELL_SIDE_SECTION"].astype(str)
    df.loc[buy_side_mask, "BUY_SIDE_SECTION"] = "buy_" + df.loc[buy_side_mask, "BUY_SIDE_SECTION"].astype(str)
    
    # Save the updated file
    df.to_csv(output_path, index=False)
    print(f"Updated file saved at: {output_path}")

# Example usage
input_file = "C:/Users/tarek/Downloads/dataset-EUR-F82CE786-DD89-11EF-9680-C7EA406D2E00/dataset-EUR-F82CE786-DD89-11EF-9680-C7EA406D2E00.xlsx"  # Change this to your actual input file path
output_file = "C:/Users/tarek/Downloads/dataset-EUR-F82CE786-DD89-11EF-9680-C7EA406D2E00/Updated_Book1.csv"  # Change this to your desired output file path
add_prefix_to_indicators(input_file, output_file)

Updated file saved at: C:/Users/tarek/Downloads/dataset-EUR-F82CE786-DD89-11EF-9680-C7EA406D2E00/Updated_Book1.csv


In [143]:
# Re-importing necessary libraries since execution state was reset
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_absolute_error

# Creating the new Sell Side DataFrame
sell_side_new_data = {
    "Gross Income_Sell": [40.636, 66.684, 54.057, 42.949, -0.829],
    "EBIT_Sell": [40.648, 66.684, 54.057, 43.522, -0.829],
    "EBITDA_Sell": [41.501, 68.237, 55.366, 44.538, -0.572],
    "Net Income_Sell": [27.04, 48.229, 41.489, 33.967, -0.876],
    "Cash & Short-Term Investments_Sell": [113.873, 117.95, 109.937, 59.103, 11.377],
    "Total Assets_Sell": [240.688, 252.334, 177.03, 145.761, 28.197],
    "Total Debt_Sell": [0, 0, 0, 0, 0],
    "Net Debt_Sell": [-113.873, -117.95, -109.937, -59.103, -11.377],
    "Total Liabilities_Sell": [202.902, 214.547, 140.75, 115.929, 10.139],
    "Total Shareholders' Equity_Sell": [37.786, 37.786, 36.28, 29.832, 18.058],
}

sell_side_new_df = pd.DataFrame(sell_side_new_data)

# Creating the new Buy Side DataFrame
buy_side_new_data = {
    "Gross Income_Buy": [-71.147, -71.596, 58.811, -178.494, -322.412],
    "EBIT_Buy": [-71.147, -71.596, 58.811, -178.494, -322.412],
    "EBITDA_Buy": [634.55, 678.226, 801.342, 578.353, 388.168],
    "Net Income_Buy": [-332.893, -479.909, -583.283, -753.911, -821.683],
    "Cash & Short-Term Investments_Buy": [976.819, 871.371, 911.627, 542.463, 201.438],
    "Total Assets_Buy": [4482.299, 4587.999, 4648.123, 4514.745, 4788.735],
    "Total Debt_Buy": [4.296, 3.237, 404.201, 621.615, 730.207],
    "Net Debt_Buy": [-972.522, -868.135, -507.425, 79.152, 528.768],
    "Total Liabilities_Buy": [8762.065, 8671.28, 8268.565, 7565.322, 7086.162],
    "Total Shareholders' Equity_Buy": [-4279.766, -4083.281, -3620.441, -3050.577, -2297.426],
}

buy_side_new_df = pd.DataFrame(buy_side_new_data)

# Extracting X (Sell Side) and y (Buy Side)
X_new = sell_side_new_df
y_new = buy_side_new_df

# Standardizing the features for better regression performance
scaler_X_new = StandardScaler()
scaler_y_new = StandardScaler()
X_new_scaled = scaler_X_new.fit_transform(X_new)
y_new_scaled = scaler_y_new.fit_transform(y_new)

# Splitting into training and testing sets (leave-one-out validation due to small dataset)
X_train_new, X_test_new, y_train_new, y_test_new = train_test_split(X_new_scaled, y_new_scaled, test_size=0.2, random_state=42)

# Training a Ridge Regression model
model_new = Ridge(alpha=1.0)
model_new.fit(X_train_new, y_train_new)

# Making predictions
y_pred_scaled_new = model_new.predict(X_test_new)

# Inversing the scaling to get real-world values
y_pred_new = scaler_y_new.inverse_transform(y_pred_scaled_new)
y_test_real_new = scaler_y_new.inverse_transform(y_test_new)

# Creating a DataFrame to compare actual vs predicted values
results_new = pd.DataFrame(y_test_real_new, columns=buy_side_new_df.columns, index=buy_side_new_df.iloc[-len(y_test_real_new):].index)
results_new["Predicted_" + results_new.columns] = y_pred_new

# Saving the results to a CSV file
results_new.to_csv("C:/Users/tarek/Downloads/actual_vs_predicted_ridge.csv", index=False)

# Evaluating the model with Mean Absolute Error
mae_values_new = mean_absolute_error(y_test_real_new, y_pred_new, multioutput='raw_values')

# Creating a DataFrame to show MAE for each indicator
mae_df_new = pd.DataFrame({"Indicator": buy_side_new_df.columns, "MAE": mae_values_new})
mae_df_new.to_csv("C:/Users/tarek/Downloads/mae_df_ridge.csv", index=False)
